In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from nltk.tokenize import TweetTokenizer
from nltk import FreqDist
from keras.preprocessing.text import Tokenizer
from sklearn import feature_extraction
import os
print(os.listdir("../input"))
import keras


# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv("../input/train.tsv",sep="\t")
test=pd.read_csv("../input/test.tsv",sep="\t")
train.columns

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.stem import SnowballStemmer,WordNetLemmatizer
stemmer=SnowballStemmer('english')
lemma=WordNetLemmatizer()


In [ ]:
def lemmize(column):
    leem=[]
    for sentence in column:
        lassos= [lemma.lemmatize(word) for word in word_tokenize(sentence)]
        lassos=' '.join(lassos)
        leem.append(lassos)
    return leem



In [ ]:
train['clean_review']=lemmize(train["Phrase"])
train.head(10)

In [ ]:
X=train.clean_review.values
y=train.Sentiment.values
y=keras.utils.to_categorical(y,num_classes=5)
from sklearn.model_selection import train_test_split
(X_train,X_val,y_train,y_val)=train_test_split(X,y,test_size=0.3,random_state=50)

In [ ]:
def distinct_words(X):
    totsent=""
    for sentence in X:
        totsent+=sentence
    alword=word_tokenize(totsent)
    return len(FreqDist(alword)) 

In [ ]:
def MAX_LENGTH(X):
    for sent in X:
      r_len=[]
      gre=word_tokenize(sent)
      r_len.append(len(gre))
    max_sent=(np.max(r_len))
    return max_sent

In [ ]:
dXt=distinct_words(X_train)
dXval=distinct_words(X_val)


In [ ]:
dXt

In [ ]:
tokenize=Tokenizer(num_words=distinct_words(X_train))
tokenize.fit_on_texts(list(X_train))
X_train=tokenize.texts_to_sequences(X_train)
X_val=tokenize.texts_to_sequences(X_val)


In [ ]:
from keras.preprocessing import sequence,text
X_train=sequence.pad_sequences(X_train,maxlen=53)
X_val=sequence.pad_sequences(X_val,maxlen=53)
X_train

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Embedding,Dense
from keras.optimizers import Adam
model1=Sequential()
model1.add(Embedding(dXt,100,mask_zero=True))
model1.add(LSTM(64,dropout=0.4, recurrent_dropout=0.4,return_sequences=True))
model1.add(LSTM(32,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model1.add(Dense(5,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

In [ ]:
history1=model1.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=5, batch_size=128, verbose=1)

In [ ]:
import matplotlib.pyplot as plt 
def plot(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
X_test=pd.read_csv("../input/test.tsv",sep='\t')
X_test['clean_review']=lemmize(X_test["Phrase"])
X_test.head(10)

In [ ]:
X_t=X_test['clean_review']
dt=distinct_words(X_t)
print

In [ ]:
X_t

In [ ]:
X_t=tokenize.texts_to_sequences(X_t)
X_t=sequence.pad_sequences(X_t,maxlen=53)
X_t

In [ ]:
prediction=model1.predict_classes(X_t,verbose=1)

In [ ]:
sub=pd.read_csv('../input/sampleSubmission.csv')
sub.Sentiment=prediction
sub

In [ ]:
sub.to_csv('lu.csv',index=False)
sub.head(5)